In [1]:
import os
import yaml
from torch.utils.data import DataLoader
import argparse

from GeospatialFM.data import get_datasets
from GeospatialFM.models import *
# from utils import load_config
from torchgeo.samplers import RandomGeoSampler
from matplotlib import pyplot as plt

from transformers import TrainingArguments, Trainer
from transformers import AdamW, get_linear_schedule_with_warmup
from GeospatialFM.utils import setup, get_eval_fn, get_data
from GeospatialFM.data import *
from GeospatialFM.models import *
from GeospatialFM.loss import *

from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import numpy as np
from torch.utils.data import ConcatDataset
import segmentation_models_pytorch as smp

%load_ext autoreload
%autoreload 2

/home/haozhesi/anaconda3/envs/sat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda:0')

In [18]:
args = {'exp_name': None,
        'config_file': 'GeospatialFM/configs/mae_cm.yaml',
        'opts': None, 
        'save_config': False}
args = argparse.Namespace(**args)
args.debug = True
args

Namespace(exp_name=None, config_file='GeospatialFM/configs/mae_cm.yaml', opts=None, save_config=False, debug=True)

In [19]:
cfg, _ = setup(args)

In [5]:
data = get_data(cfg)
data

{'train': DataInfo(dataloader=<torch.utils.data.dataloader.DataLoader object at 0x7f272e84c280>, sampler=None, shared_epoch=None),
 'val': DataInfo(dataloader=<torch.utils.data.dataloader.DataLoader object at 0x7f272e84c190>, sampler=None, shared_epoch=None),
 'test': DataInfo(dataloader=<torch.utils.data.dataloader.DataLoader object at 0x7f272e84c5e0>, sampler=None, shared_epoch=None)}

In [6]:
sample = iter(data['train'].dataloader).__next__()

In [7]:
sample['image'].shape

torch.Size([512, 13, 224, 224])

In [8]:
model = construct_mae(cfg.MODEL)

In [9]:
device = 'cuda:0'
device_ids = [0, 1, 2, 3]

In [10]:
model = model.to(device)
model = torch.nn.DataParallel(model, device_ids=device_ids)

In [20]:
loss = get_loss_list(cfg.LOSS)

In [21]:
loss

[ClipLoss(), MAELoss()]

In [11]:
device = torch.device(device)

In [12]:
image = sample['image'].to(device, non_blocking=True)
radar = sample['radar'].to(device, non_blocking=True)

In [13]:
model_out = model(image, radar)
logit_scale = model_out.get("logit_scale").mean()
model_out['logit_scale'] = logit_scale

/home/haozhesi/anaconda3/envs/sat/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [48]:
mae_loss = MAELoss()
mmce_loss = MultiModalCELoss()

In [47]:
mae_loss(**ret, return_dict=True)

{'optical_mse': tensor(0.8039, grad_fn=<DivBackward0>),
 'radar_mse': tensor(0.8062, grad_fn=<DivBackward0>)}

In [51]:
mmce_loss(**ret, labels=sample['label'], return_dict=True)


{'optical_ce': tensor(2.9425, grad_fn=<NllLossBackward0>),
 'radar_ce': tensor(2.8541, grad_fn=<NllLossBackward0>)}